# MCP Local Demo (Stdio Transport)

This notebook demonstrates connecting to a **local MCP server** over stdio, in contrast to the remote Streamable HTTP transport in d2r.

Local MCP servers run as subprocesses on your machine, communicating via stdin/stdout. They can access **local resources** that remote servers cannot: the filesystem, system clock, local databases, and more.

We'll use a small Python MCP server (`d2l-local-mcp/server.py`) that provides a `get_current_datetime` tool.

In [ ]:
import json
import os
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(
    command="uv",
    args=["run", "python", os.path.abspath("d2l-local-mcp/server.py")],
)

In [ ]:
async def list_local_tools():
    """Connect to local MCP server and list available tools."""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            return await session.list_tools()

tools_response = await list_local_tools()

print(f"Found {len(tools_response.tools)} tool(s):\n")
for tool in tools_response.tools:
    print(f"- {tool.name}")
    print(f"  {tool.description}\n")

In [ ]:
async def call_local_tool(tool_name: str, args: dict):
    """Execute a tool on the local MCP server."""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            return await session.call_tool(tool_name, arguments=args)

result = await call_local_tool("get_current_datetime", {})

for content in result.content:
    if hasattr(content, 'text'):
        print(content.text)

## Agentic Pattern: LLM + MCP Tools

The previous cells called MCP tools directly with hardcoded parameters. Now we'll let **Claude** decide which tools to call and how to interpret the results.

This is the core agentic pattern:
1. Send a question and available tools to Claude
2. Claude decides which tool(s) to call and with what parameters
3. We execute the tool calls via MCP and feed results back
4. Claude reasons over the data and provides a final answer

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv(override=True)

anthropic_client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

# Convert MCP tools to Claude's tool format
claude_tools = []
for tool in tools_response.tools:
    claude_tools.append({
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    })

print(f"Registered {len(claude_tools)} tool(s) for Claude")

claude_tools

## Ask Claude: What Time Is It?

Claude doesn't have access to the current time on its own. With the local MCP tool, it can query the system clock — something only a local server can do.

In [ ]:
async def ask_claude_with_mcp(question: str):
    """Send a question to Claude, letting it use local MCP tools."""
    print(f"Question: {question}\n")

    messages = [{"role": "user", "content": question}]

    response = anthropic_client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        tools=claude_tools,
        messages=messages
    )

    # Agentic loop: keep going while Claude wants to use tools
    while response.stop_reason == "tool_use":
        tool_use_blocks = [block for block in response.content if block.type == "tool_use"]

        if not tool_use_blocks:
            break

        messages.append({"role": "assistant", "content": response.content})

        # Execute each tool call via the local MCP server
        tool_results = []
        for tool_block in tool_use_blocks:
            tool_name = tool_block.name
            tool_args = tool_block.input

            print(f"  Tool call: {tool_name}")
            print(f"  Arguments: {json.dumps(tool_args, indent=4)}")

            result = await call_local_tool(tool_name, tool_args)

            content_text = ""
            for item in result.content:
                if hasattr(item, 'text'):
                    content_text += item.text

            print(f"  Result: {content_text}\n")

            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_block.id,
                "content": content_text
            })

        # Feed tool results back to Claude
        messages.append({"role": "user", "content": tool_results})

        response = anthropic_client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            tools=claude_tools,
            messages=messages
        )

    for block in response.content:
        if hasattr(block, 'text'):
            print(block.text)

await ask_claude_with_mcp(
    "What is the current date and time? What day of the week is it?"
)